In [1]:
%run /home/cz136/project/sa/lib/header.ipynb

In [26]:
import treecorr

In [74]:
def get_ng(cluster):
    
    def get_cluster_cen(cluster):
        cen_id=cluster[('Alt','Alt1','ID_CENT')]
        return (shapes.loc[cen_id])
    
    def get_cluster_sats(cluster):
        cen_mat_id=cluster.name
        sats=shapes[shapes['All','MEM_MATCH_ID']==cen_mat_id]
        return (sats)
    
    cen=get_cluster_cen(cluster)
    sats=get_cluster_sats(cluster)
    
    
    sats_e1=sats[('All','e1')].to_numpy()
    sats_e2=sats[('All','e2')].to_numpy()
    
#     print(sats_e1,sats_e2)
    
    cen_e1=cen[('All','e1')].reshape(1)
    cen_e2=cen[('All','e2')].reshape(1)
    
#     print(cen_e1,cen_e2)
    
    sats_ra=sats[('All','RA')].to_numpy()
    sats_dec=sats[('All','DEC')].to_numpy()
    
    cen_ra=np.array(cen[('All','RA')]).reshape(1)
    cen_dec=np.array(cen[('All','DEC')]).reshape(1)
    
    
#     print(np.shape(cen_ra),np.shape(cen_dec))    
#     print(cen_ra,cen_dec)    
        
    sats_cat = treecorr.Catalog( g1 = sats_e1, g2   = sats_e2, 
                                 ra = sats_ra, dec = sats_dec, 
                                 ra_units='deg', dec_units='deg')
    
    cen_cat = treecorr.Catalog( g1 = cen_e1, g2   = cen_e2, 
                                 ra = cen_ra, dec = cen_dec, 
                                 ra_units='deg', dec_units='deg')
    
    ng = treecorr.NGCorrelation(nbins=20, min_sep=0.1, max_sep=10, sep_units='arcmin', bin_slop=0.01,bin_type="Linear")
    ng.process(cen_cat,sats_cat)
    
    
    return(ng)

get_ng(clusters.iloc[0]).xi

array([ 3.20921049e-02,  1.77562343e+03, -3.76407011e+02, -2.59644098e-02,
        1.98680676e-02,  2.19478034e-02,  1.09553932e+02,  2.57535321e+02,
       -9.42034719e+01,  2.15897192e+02, -1.03044844e-02,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00])

In [19]:
test_id=clusters.iloc[0][('Alt','Alt1','ID_CENT')]

In [53]:
test_members=shapes[shapes[('All','MEM_MATCH_ID')]==1]

In [21]:
test_center=shapes.loc[test_id]

In [24]:
test_center

Band  Data           
All   MEM_MATCH_ID              1
      Z                  0.429804
      RA                  43.5646
      DEC                 -58.953
      R                         0
                           ...   
      flags_select_2p           2
      flags_select_2m           2
      PP                        1
      Theta_cen                90
      Theta_sat               -90
Name: 3089939451, Length: 112, dtype: object